In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler
import os

import shutil

from PIL import Image
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Використовуємо пристрій: {device}")
if device.type == 'cuda':
    print(f"Назва відеокарти: {torch.cuda.get_device_name(0)}")

data_dir = 'fast_data'
train_dir = os.path.join(data_dir, 'train')

Використовуємо пристрій: cuda
Назва відеокарти: Tesla T4


In [ ]:

os.chdir('/content')


print("Поточна папка:", os.getcwd())
print("Вміст папки:", os.listdir())

Поточна папка: /content/drive/MyDrive/butterfly_project
Вміст папки: ['dataset', 'final_butterfly_model.pth']


In [ ]:

classes_path = os.path.join(data_dir, 'classes.txt')

with open(classes_path, 'r') as f:

    class_names = [line.strip() for line in f.readlines()]

num_classes = len(class_names)

print(f"Кількість класів: {num_classes}")
print(f"Перші 5 класів для прикладу: {class_names[:5]}")

Кількість класів: 100
Перші 5 класів для прикладу: ['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO']


In [ ]:

train_dir = '/content/fast_data/train'


data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms)


train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,       
    shuffle=True,
    num_workers=2,       
    pin_memory=True      
)

print(f"Дані завантажено з: {train_dir}")
print(f"Класів: {len(train_dataset.classes)}")

Дані завантажено з: /content/local_train
Класів: 100


In [ ]:
"""

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),          
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])


train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms)


batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

print(f"Знайдено зображень: {len(train_dataset)}")
"""

'\n# Трансформації для зображень (підготовка під AlexNet)\ndata_transforms = transforms.Compose([\n    transforms.Resize((224, 224)),  # AlexNet хоче саме цей розмір\n    transforms.ToTensor(),          # Перетворюємо картинку в цифри (тензор)\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], # Стандартні числа для ImageNet моделей\n                         std=[0.229, 0.224, 0.225])\n])\n\n# Завантажуємо тренувальний датасет\ntrain_dataset = datasets.ImageFolder(train_dir, transform=data_transforms)\n\n# Створюємо DataLoader (він буде подавати дані батчами, тобто пачками)\nbatch_size = 32\ntrain_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)\n\nprint(f"Знайдено зображень: {len(train_dataset)}")\n'

In [ ]:

print("Завантажуємо AlexNet...")
model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)


for param in model.features.parameters():
    param.requires_grad = False


print("Старий класифікатор:", model.classifier[6])

model.classifier[6] = nn.Linear(4096, num_classes)

model = model.to(device)

print("Новий класифікатор:", model.classifier[6])

Завантажуємо AlexNet...
Старий класифікатор: Linear(in_features=4096, out_features=1000, bias=True)
Новий класифікатор: Linear(in_features=4096, out_features=100, bias=True)


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

epochs = 30

print("\n--- ПОЧИНАЄМО ТРЕНУВАННЯ (ADAM + SCHEDULER) ---")

for epoch in range(epochs):
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0


    model.train()

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)


        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)


        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_preds += labels.size(0)
        correct_preds += (predicted == labels).sum().item()


    exp_lr_scheduler.step()


    current_lr = optimizer.param_groups[0]['lr']

    epoch_acc = 100 * correct_preds / total_preds
    epoch_loss = running_loss / len(train_loader)

    print(f"🏁 Епоха [{epoch+1}/{epochs}] | "
          f"Loss: {epoch_loss:.4f} | "
          f"Acc: {epoch_acc:.2f}% | "
          f"LR: {current_lr:.6f}")

print("Тренування завершено!")


--- ПОЧИНАЄМО ТРЕНУВАННЯ (ADAM + SCHEDULER) ---
🏁 Епоха [1/30] | Loss: 1.5729 | Acc: 60.69% | LR: 0.001000
🏁 Епоха [2/30] | Loss: 0.8270 | Acc: 77.57% | LR: 0.000100
🏁 Епоха [3/30] | Loss: 0.3493 | Acc: 89.52% | LR: 0.000100
🏁 Епоха [4/30] | Loss: 0.2155 | Acc: 93.39% | LR: 0.000010
🏁 Епоха [5/30] | Loss: 0.1669 | Acc: 94.48% | LR: 0.000010
🏁 Епоха [6/30] | Loss: 0.1509 | Acc: 95.30% | LR: 0.000001
🏁 Епоха [7/30] | Loss: 0.1497 | Acc: 95.13% | LR: 0.000001
🏁 Епоха [8/30] | Loss: 0.1415 | Acc: 95.33% | LR: 0.000000
🏁 Епоха [9/30] | Loss: 0.1481 | Acc: 95.32% | LR: 0.000000
🏁 Епоха [10/30] | Loss: 0.1463 | Acc: 95.42% | LR: 0.000000
🏁 Епоха [11/30] | Loss: 0.1450 | Acc: 95.28% | LR: 0.000000
🏁 Епоха [12/30] | Loss: 0.1438 | Acc: 95.38% | LR: 0.000000
🏁 Епоха [13/30] | Loss: 0.1447 | Acc: 95.18% | LR: 0.000000
🏁 Епоха [14/30] | Loss: 0.1452 | Acc: 95.34% | LR: 0.000000
🏁 Епоха [15/30] | Loss: 0.1441 | Acc: 95.48% | LR: 0.000000
🏁 Епоха [16/30] | Loss: 0.1460 | Acc: 95.34% | LR: 0.000000


In [ ]:
save_path = '/content/drive/MyDrive/butterfly_project/final_butterfly_model.pth'


torch.save(model.state_dict(), save_path)

print(f"✅ Готово! Модель збережено у файл: {save_path}")

✅ Готово! Модель збережено у файл: /content/drive/MyDrive/butterfly_project/final_butterfly_model.pth


In [ ]:

load_path = '/content/drive/MyDrive/butterfly_project/final_butterfly_model.pth'


loaded_model = models.alexnet(weights=None)

loaded_model.classifier[6] = nn.Linear(4096, num_classes)


loaded_model.load_state_dict(torch.load(load_path, map_location=device))

loaded_model.eval()
loaded_model.to(device)

print("✅ Модель успішно відновлено і вона готова до роботи!")

✅ Модель успішно відновлено і вона готова до роботи!


In [ ]:

class ButterflyTestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform

        self.image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)


        image = Image.open(img_path).convert("RGB")


        if self.transform:
            image = self.transform(image)


        return image, img_name


test_dir = '/content/fast_data/test'

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = ButterflyTestDataset(test_dir, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) 

print(f"Знайдено тестових зображень: {len(test_dataset)}")

Знайдено тестових зображень: 2728


In [ ]:

try:
    class_names = train_dataset.classes
except NameError:
    print("Відновлюємо список класів...")
    temp_data = datasets.ImageFolder('/content/fast_data/train')
    class_names = temp_data.classes

print(f"Всього класів: {len(class_names)}")


all_ids = []
all_preds = []

loaded_model.eval() 
loaded_model.to(device)

print("⏳ Починаю передбачення...")

with torch.no_grad(): 
    for inputs, filenames in tqdm(test_loader):
        inputs = inputs.to(device)


        outputs = loaded_model(inputs)


        _, predicted_indices = torch.max(outputs, 1)


        for i in range(len(filenames)):
            file_id = os.path.splitext(filenames[i])[0]
            all_ids.append(file_id)

            class_idx = predicted_indices[i].item()
            predicted_label = class_names[class_idx]
            all_preds.append(predicted_label)

submission_df = pd.DataFrame({
    'id': all_ids,
    'label': all_preds
})

csv_path = '/content/drive/MyDrive/butterfly_project/submission.csv'
submission_df.to_csv(csv_path, index=False)

print(f"\n✅ Готово! Файл збережено: {csv_path}")
print("Ось перші 5 рядків для перевірки:")
print(submission_df.head())

Відновлюємо список класів...
Всього класів: 100
⏳ Починаю передбачення...


100%|██████████| 86/86 [00:06<00:00, 13.84it/s]


✅ Готово! Файл збережено: /content/drive/MyDrive/butterfly_project/submission.csv
Ось перші 5 рядків для перевірки:
                                 id              label
0  7bf8001f46fd4b709a367ddfa91d4971  COMMON WOOD-NYMPH
1  d0058361d71b4cb4a498d4154d3cf5ff             ADONIS
2  a8be47bb65c2463d9e9f9f291e6dce09            MONARCH
3  738c1ed1771345af87465ce9868edb4b   GREEN HAIRSTREAK
4  7d8c94c8ad014295be46844983b76b87        RED CRACKER
